In [1]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-5257U CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, broadwell)


# Optimal Policies with Dynamic Programming

Welcome to Assignment 2. This notebook will help you understand:
- Policy Evaluation and Policy Improvement.
- Value and Policy Iteration.
- Bellman Equations.

## Gridworld City

Gridworld City, a thriving metropolis with a booming technology industry, has recently experienced an influx of grid-loving software engineers. Unfortunately, the city's street parking system, which charges a fixed rate, is struggling to keep up with the increased demand. To address this, the city council has decided to modify the pricing scheme to better promote social welfare. In general, the city considers social welfare higher when more parking is being used, the exception being that the city prefers that at least one spot is left unoccupied (so that it is available in case someone really needs it). The city council has created a Markov decision process (MDP) to model the demand for parking with a reward function that reflects its preferences. Now the city has hired you &mdash; an expert in dynamic programming &mdash; to help determine an optimal policy.

In the city council's parking MDP, states are nonnegative integers indicating how many parking spaces are occupied, actions are nonnegative integers designating the price of street parking, the reward is a real value describing the city's preference for the situation, and time is discretized by hour. As might be expected, charging a high price is likely to decrease occupancy over the hour, while charging a low price is likely to increase it.

For now, let's consider an environment with three parking spaces and three price points. Note that an environment with three parking spaces actually has four states &mdash; zero, one, two, or three spaces could be occupied.

In [2]:
using StatsBase
using PyCall

mutable struct ParkingWorld
    num_spaces::Int64
    num_prices::Int64
    price_factor::Float64
    occupants_factor::Float64
    null_factor::Float64
    State::Array{Int64,1}
    Action::Array{Int64,1}
    function ParkingWorld(;num_spaces = 10, num_prices = 4, price_factor = 0.1,
                          occupants_factor = 1.0, null_factor = 1/3)
        parkingworld = new()
        parkingworld.num_spaces = num_spaces
        parkingworld.num_prices = num_prices
        parkingworld.price_factor = price_factor
        parkingworld.null_factor = 1/3
        parkingworld.occupants_factor = occupants_factor
        parkingworld.State = [num_occupied for num_occupied in 0:num_spaces]
        parkingworld.Action = collect(0:num_prices-1)
        parkingworld
    end
end



function policy(park::ParkingWorld, state_, reward, state, action)
    if reward != get_reward(park, state, state_)
        return 0
    else
        center = (1 - park.price_factor) * state +
                    park.price_factor * park.num_spaces * (1 - action/park.num_prices)
        emphasis = exp.(-abs.(range(0, stop = (2 * park.num_spaces-1)) .- center) ./ 5)
        if state_ == park.num_spaces
            return sum(emphasis[state_+1:end]) / sum(emphasis)
        end
        return emphasis[state_+1] / sum(emphasis)
    end
end


function support(park::ParkingWorld, state, action)
    return [(state_, get_reward(park, state, state_)) for state_ in park.State]
end

function transitions(park::ParkingWorld, state, action)
    trans = [[reward, policy(park, state_, reward, state, action)]
                for (state_, reward) in support(park, state, action)]
    tmp = trans[1]
    for i in 2:length(trans)
        tmp = hcat(tmp, trans[i])
    end
    trans = tmp'
    return trans
end


function get_reward(park::ParkingWorld, state, state_)
    return state_reward(park, state) + state_reward(park, state_)
end

function state_reward(park::ParkingWorld, state)
    if state == park.num_spaces
        return park.null_factor * state * park.occupants_factor
    else
        return state * park.occupants_factor
    end
end

function random_state(park::ParkingWorld)
    return rand(0:(park.num_prices-1))
end


function step_(park::ParkingWorld, state, action)
    probabilities = [
        policy(park, state_, reward(park, state, state_), state, action) for state_ in park.State
    ]
    return sample(park.State, Weights(probabilities))
end


step_ (generic function with 1 method)

In [3]:
num_spaces = 3
num_prices = 3
env = ParkingWorld(num_spaces = num_spaces, num_prices = num_prices)

# The value function is a one-dimensional array where the  i-th entry gives the value of  i
# spaces being occupied.
V = zeros(num_spaces + 1)
pi_array = ones(num_spaces + 1, num_prices) ./ num_prices

state = 1
V[state] = 10
pi_array[1,:] = [0.75, 0.21, 0.04]

3-element Array{Float64,1}:
 0.75
 0.21
 0.04

In [4]:
pi_array

4×3 Array{Float64,2}:
 0.75      0.21      0.04
 0.333333  0.333333  0.333333
 0.333333  0.333333  0.333333
 0.333333  0.333333  0.333333

In [5]:
for i in 1:size(pi_array,1)
    tmp = pi_array[i, :]
    println(join("pi(A=$a|S=$(i-1)) = $(round(p, digits=2))" * repeat(" ",4) for (a, p) in enumerate(tmp)))
end

pi(A=1|S=0) = 0.75    pi(A=2|S=0) = 0.21    pi(A=3|S=0) = 0.04    
pi(A=1|S=1) = 0.33    pi(A=2|S=1) = 0.33    pi(A=3|S=1) = 0.33    
pi(A=1|S=2) = 0.33    pi(A=2|S=2) = 0.33    pi(A=3|S=2) = 0.33    
pi(A=1|S=3) = 0.33    pi(A=2|S=3) = 0.33    pi(A=3|S=3) = 0.33    


In [6]:
np = pyimport("numpy")
plt = pyimport("matplotlib.pyplot")
ticker = pyimport("matplotlib.ticker")
axes_grid1 = pyimport("mpl_toolkits.axes_grid1")

PyObject <module 'mpl_toolkits.axes_grid1' from '/Users/swami/.julia/Conda_env/lib/python3.8/site-packages/mpl_toolkits/axes_grid1/__init__.py'>

In [7]:
plt.rc("font", size=30)  # controls default text sizes
plt.rc("axes", titlesize=25)  # fontsize of the axes title
plt.rc("axes", labelsize=25)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=17)  # fontsize of the tick labels
plt.rc("ytick", labelsize=17)  # fontsize of the tick labels
plt.rc("legend", fontsize=20)  # legend fontsize
plt.rc("figure", titlesize=30)
plt.tight_layout()

In [8]:
function plot_tool(V, pi)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12.5, 5))
    ax1.axis("on")
    ax1.cla()
    states = np.arange(size(V, 1))
    ax1.bar(states, V, edgecolor="none")
    ax1.set_xlabel("State")
    ax1.set_ylabel("Value", rotation="horizontal", ha="right")
    ax1.set_title("Value Function")
    ax1.xaxis.set_major_locator(ticker.MaxNLocator(integer=true, nbins=6))
    ax1.yaxis.grid()
    ax1.set_ylim(bottom=minimum(V))
    # plot policy
    ax2.axis("on")
    ax2.cla()
    im = ax2.imshow(pi', cmap="Greys", vmin=0, vmax=1, aspect="auto")
    ax2.invert_yaxis()
    ax2.set_xlabel("State")
    ax2.set_ylabel("Action", rotation="horizontal", ha="right")
    ax2.set_title("Policy")
    start, last = ax2.get_xlim()
    ax2.xaxis.set_ticks(np.arange(start, last), minor=true)
    ax2.xaxis.set_major_locator(ticker.MaxNLocator(integer=true, nbins=6))
    ax2.yaxis.set_major_locator(ticker.MaxNLocator(integer=true, nbins=6))
    start, last = ax2.get_ylim()
    ax2.yaxis.set_ticks(np.arange(start, last), minor=true)
    ax2.grid(which="minor")
    divider = axes_grid1.make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="5%", pad=0.20)
    cbar = fig.colorbar(im, cax=cax, orientation="vertical")
    cbar.set_label("Probability", rotation=0, ha="left")
    fig.subplots_adjust(wspace=0.5)
    sleep(0.001)
    plt.show()
    plt.close()
end

plot_tool (generic function with 1 method)

In [ ]:
plot_tool(V, pi_array)

On the left, the value function is displayed as a barplot. State zero has an expected return of ten, while the other states have an expected return of zero. On the right, the policy is displayed on a two-dimensional grid. Each vertical strip gives the policy at the labeled state. In state zero, action zero is the darkest because the agent's policy makes this choice with the highest probability. In the other states the agent has the equiprobable policy, so the vertical strips are colored uniformly.

In [9]:
state = 3
action = 1

transition = transitions(env, state, action)


for i in 1:size(transition,1)
    tmp = transition[i, :]
    println("p(S\'=$i, R=$(tmp[1]) | S=$state, A = $action)=$(round(tmp[2], digits=2))")
end

p(S'=1, R=1.0 | S=3, A = 1)=0.12
p(S'=2, R=2.0 | S=3, A = 1)=0.15
p(S'=3, R=3.0 | S=3, A = 1)=0.18
p(S'=4, R=2.0 | S=3, A = 1)=0.54


## Section 1: Policy Evaluation

First, the city council would like you to evaluate the quality of the existing pricing scheme. Policy evaluation works by iteratively applying the Bellman equation for $v_{\pi}$ to a working value function, as an update rule, as shown below.

$$\large v(s) \leftarrow \sum_a \pi(a | s) \sum_{s', r} p(s', r | s, a)[r + \gamma v(s')]$$
This update can either occur "in-place" (i.e. the update rule is sequentially applied to each state) or with "two-arrays" (i.e. the update rule is simultaneously applied to each state). Both versions converge to $v_{\pi}$ but the in-place version usually converges faster. **In this assignment, we will be implementing all update rules in-place**, as is done in the pseudocode of chapter 4 of the textbook. 

We have written an outline of the policy evaluation algorithm described in chapter 4.1 of the textbook. It is left to you to fill in the `bellman_update` function to complete the algorithm.

In [10]:
function evaluate_policy(env::ParkingWorld, V, pi_array, gamma, theta)
    delta = Inf
    while delta > theta
        delta = 0
        for s in env.State
            v = V[s+1]
            bellman_update(env, V, pi_array, s, gamma)
            delta = max(delta, abs(v - V[s+1]))
        end
    end
    return V
end

evaluate_policy (generic function with 1 method)

In [11]:
function bellman_update(env::ParkingWorld, V, pi_array, s, gamma)
    # Mutate V according to the Bellman update equation
    state_value = 0
    for action in env.Action
        trans = transitions(env, s, action) # trainsition value
        state_value_tmp = 0
        for state in env.State
            state_value_tmp += trans[state+1, 2] * (trans[state+1,1] +
                                                    gamma * V[state+1])
        end
        state_value += pi_array[s+1, action+1] * state_value_tmp
    end
    V[s+1] = state_value
end

bellman_update (generic function with 1 method)

The cell below uses the policy evaluation algorithm to evaluate the city's policy, which charges a constant price of one.

In [12]:
num_spaces = 10
num_prices = 4
env = ParkingWorld(num_spaces = num_spaces, num_prices=num_prices)
V = zeros(num_spaces+1)
city_policy = zeros(num_spaces+1, num_prices)
city_policy[:,2].=1
gamma = 0.9
theta = 0.1
V = evaluate_policy(env, V, city_policy, gamma, theta);

In [14]:
env.State

11-element Array{Int64,1}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [15]:
env.Action

4-element Array{Int64,1}:
 0
 1
 2
 3

In [13]:
V

11-element Array{Float64,1}:
 80.04173398685609
 81.65532302729444
 83.37394007142157
 85.12975565977709
 86.87174913186462
 88.55589131273771
 90.14020421970433
 91.58180605347238
 92.81929841429525
 93.78915889368793
 87.7779299121454

In [16]:
city_policy

11×4 Array{Float64,2}:
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0

In [ ]:
plot_tool(V, city_policy)

You can check the output (rounded to one decimal place) against the answer below:<br>
State $\quad\quad$    Value<br>
0 $\quad\quad\quad\;$        80.0<br>
1 $\quad\quad\quad\;$        81.7<br>
2 $\quad\quad\quad\;$        83.4<br>
3 $\quad\quad\quad\;$        85.1<br>
4 $\quad\quad\quad\;$        86.9<br>
5 $\quad\quad\quad\;$        88.6<br>
6 $\quad\quad\quad\;$        90.1<br>
7 $\quad\quad\quad\;$        91.6<br>
8 $\quad\quad\quad\;$        92.8<br>
9 $\quad\quad\quad\;$        93.8<br>
10 $\quad\quad\;\;\,\,$       87.8<br>

Observe that the value function qualitatively resembles the city council's preferences &mdash; it monotonically increases as more parking is used, until there is no parking left, in which case the value is lower. Because of the relatively simple reward function (more reward is accrued when many but not all parking spots are taken and less reward is accrued when few or all parking spots are taken) and the highly stochastic dynamics function (each state has positive probability of being reached each time step) the value functions of most policies will qualitatively resemble this graph. However, depending on the intelligence of the policy, the scale of the graph will differ. In other words, better policies will increase the expected return at every state rather than changing the relative desirability of the states. Intuitively, the value of a less desirable state can be increased by making it less likely to remain in a less desirable state. Similarly, the value of a more desirable state can be increased by making it more likely to remain in a more desirable state. That is to say, good policies are policies that spend more time in desirable states and less time in undesirable states. As we will see in this assignment, such a steady state distribution is achieved by setting the price to be low in low occupancy states (so that the occupancy will increase) and setting the price high when occupancy is high (so that full occupancy will be avoided).

## Section 2: Policy Iteration
Now the city council would like you to compute a more efficient policy using policy iteration. Policy iteration works by alternating between evaluating the existing policy and making the policy greedy with respect to the existing value function. We have written an outline of the policy iteration algorithm described in chapter 4.3 of the textbook. We will make use of the policy evaluation algorithm you completed in section 1. It is left to you to fill in the `q_greedify_policy` function, such that it modifies the policy at $s$ to be greedy with respect to the q-values at $s$, to complete the policy improvement algorithm.

In [17]:
function improve_policy(env::ParkingWorld, V, pi_array, gamma)
    policy_stable = true
    for s in env.State
        old = copy(pi_array[s+1])
        q_greedify_policy(env, V, pi_array, s, gamma)
        if !(isequal(pi_array[s+1], old))
            policy_stable = false
        end
    end
    return pi_array, policy_stable
end

improve_policy (generic function with 1 method)

In [18]:
function policy_iteration(env::ParkingWorld, gamma, theta)
    V = zeros(length(env.State))
    pi_array = ones(length(env.State), length(env.Action)) / length(env.Action)
    policy_stable = false
    while !policy_stable
        V = evaluate_policy(env, V, pi_array, gamma, theta)
        pi_array, policy_stable = improve_policy(env, V, pi_array, gamma)
    end
    return V, pi_array
end

policy_iteration (generic function with 1 method)

In [19]:
function q_greedify_policy(env::ParkingWorld, V, pi_array, s, gamma)
    """Mutate pi_array to be greedy with respect to the q-values induced by V."""
    # update pi_array
    pi_value = zeros(length(env.Action)) # for calculate the argmax
    for action in env.Action
        trans = transitions(env, s, action) # trainsition value
        for state in env.State
            pi_value[action+1] = pi_value[action+1] + trans[state + 1, 2] *
                                 (trans[state+1, 1] + gamma * V[state+1])
        end
    end
    action_max = argmax(pi_value)
    # pi_array: m*n matrix, m represent different state, n represent different
    # action respect to different state
    pi_array[s+1,:] = zeros(size(pi_array, 2))
    # represent 100% probability to take one action locate in action_max
    pi_array[s+1, action_max] = 1
end

q_greedify_policy (generic function with 1 method)

In [20]:
env = ParkingWorld(num_spaces=10, num_prices=4)
gamma = 0.9
theta = 0.1
V, pi_array = policy_iteration(env, gamma, theta);

In [21]:
V

11-element Array{Float64,1}:
 81.60940116840347
 83.28357754451835
 85.03018628050616
 86.79007707441114
 88.51662022628072
 90.16819234877124
 91.70422112901124
 93.08268939887775
 94.25817122718998
 95.25809637574687
 89.45397248620606

In [22]:
pi_array

11×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0

In [ ]:
plot_tool(V, pi_array)

You can check the value function (rounded to one decimal place) and policy against the answer below:<br>
State $\quad\quad$    Value $\quad\quad$ Action<br>
0 $\quad\quad\quad\;$        81.6 $\quad\quad\;$ 0<br>
1 $\quad\quad\quad\;$        83.3 $\quad\quad\;$ 0<br>
2 $\quad\quad\quad\;$        85.0 $\quad\quad\;$ 0<br>
3 $\quad\quad\quad\;$        86.8 $\quad\quad\;$ 0<br>
4 $\quad\quad\quad\;$        88.5 $\quad\quad\;$ 0<br>
5 $\quad\quad\quad\;$        90.2 $\quad\quad\;$ 0<br>
6 $\quad\quad\quad\;$        91.7 $\quad\quad\;$ 0<br>
7 $\quad\quad\quad\;$        93.1 $\quad\quad\;$ 0<br>
8 $\quad\quad\quad\;$        94.3 $\quad\quad\;$ 0<br>
9 $\quad\quad\quad\;$        95.3 $\quad\quad\;$ 4<br>
10 $\quad\quad\;\;\,\,$      89.5 $\quad\quad\;$ 4<br>

## Section 3: Value Iteration
The city has also heard about value iteration and would like you to implement it. Value iteration works by iteratively applying the Bellman optimality equation for $v_{\ast}$ to a working value function, as an update rule, as shown below.

$$\large v(s) \leftarrow \max_a \sum_{s', r} p(s', r | s, a)[r + \gamma v(s')]$$
We have written an outline of the value iteration algorithm described in chapter 4.4 of the textbook. It is left to you to fill in the `bellman_optimality_update` function to complete the value iteration algorithm.

In [23]:
function value_iteration(env::ParkingWorld, gamma, theta)
    V = zeros(length(env.State))
    while true
        delta = 0
        for s in env.State
            v = V[s+1] 
            bellman_optimality_update(env, V, s, gamma) 
            delta = max(delta, abs(v-  V[s+1]))　
        end
        if delta < theta 
            break
        end
    end
    pi_array = ones(length(env.State), length(env.Action)) / length(env.Action)
    for s in env.State
        q_greedify_policy(env, V, pi_array, s, gamma)
    end
    return V, pi_array
end

value_iteration (generic function with 1 method)

In [24]:
function bellman_optimality_update(env::ParkingWorld, V, s, gamma)
    """Mutate `V` according to the Bellman optimality update equation"""
    # initial a series to get the max of the sum
    state_value = zeros(length(env.Action))
    for action in env.Action
        trans = transitions(env, s, action)
        for state in env.State
            state_value[action+1] = state_value[action+1] + trans[state+1,2] *
                                    (trans[state+1, 1] + gamma * V[state+1])
        end
    end
    V[s+1] = maximum(state_value)
end

bellman_optimality_update (generic function with 1 method)

In [25]:
env = ParkingWorld(num_spaces=10, num_prices=4)
gamma = 0.9
theta = 0.1
V, pi_array = value_iteration(env, gamma, theta);

In [26]:
V

11-element Array{Float64,1}:
 81.60486890609062
 83.27914232401172
 85.02582906625796
 86.78578527433001
 88.51238499666755
 90.16400717647323
 91.70008102620021
 93.07859041796065
 94.25411015493282
 95.2541025364668
 89.45001307760359

In [27]:
pi_array

11×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0

You can check your value function (rounded to one decimal place) and policy against the answer below:<br>
State $\quad\quad$    Value $\quad\quad$ Action<br>
0 $\quad\quad\quad\;$        81.6 $\quad\quad\;$ 0<br>
1 $\quad\quad\quad\;$        83.3 $\quad\quad\;$ 0<br>
2 $\quad\quad\quad\;$        85.0 $\quad\quad\;$ 0<br>
3 $\quad\quad\quad\;$        86.8 $\quad\quad\;$ 0<br>
4 $\quad\quad\quad\;$        88.5 $\quad\quad\;$ 0<br>
5 $\quad\quad\quad\;$        90.2 $\quad\quad\;$ 0<br>
6 $\quad\quad\quad\;$        91.7 $\quad\quad\;$ 0<br>
7 $\quad\quad\quad\;$        93.1 $\quad\quad\;$ 0<br>
8 $\quad\quad\quad\;$        94.3 $\quad\quad\;$ 0<br>
9 $\quad\quad\quad\;$        95.3 $\quad\quad\;$ 4<br>
10 $\quad\quad\;\;\,\,$      89.5 $\quad\quad\;$ 4<br>

In [ ]:
plot_tool(V, pi_array)

In the value iteration algorithm above, a policy is not explicitly maintained until the value function has converged. Below, we have written an identically behaving value iteration algorithm that maintains an updated policy. Writing value iteration in this form makes its relationship to policy iteration more evident. Policy iteration alternates between doing complete greedifications and complete evaluations. On the other hand, value iteration alternates between doing local greedifications and local evaluations. 

In [28]:
function value_iteration2(env::ParkingWorld, gamma, theta)
    V = zeros(length(env.State))
    pi_array = ones(length(env.State), length(env.Action)) ./ length(env.Action)
    while true
        delta = 0
        for s in env.State
            v = V[s+1]
            q_greedify_policy(env, V, pi_array, s, gamma)
            bellman_update(env, V, pi_array, s, gamma)
            delta = max(delta, abs(v - V[s+1]))
        end
        if delta < theta
            break
        end
    end
    return V, pi_array
end

value_iteration2 (generic function with 1 method)

In [29]:
env = ParkingWorld(num_spaces=10, num_prices=4)
gamma = 0.9
theta = 0.1
V, pi_array = value_iteration2(env, gamma, theta);

In [30]:
V

11-element Array{Float64,1}:
 81.60486890609062
 83.27914232401172
 85.02582906625796
 86.78578527433001
 88.51238499666755
 90.16400717647323
 91.70008102620021
 93.07859041796065
 94.25411015493282
 95.2541025364668
 89.45001307760359

In [31]:
pi_array

11×4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0

In [ ]:
plot_tool(V, pi_array)